In [ ]:
save("newV.jld", "data", ls.V)
save("newV0.jld", "data", ls.V0)
save("newVb.jld", "data", ls.Vb)
save("newpolicyv.jld", "data", ls.policy_v)
save("newpolicyv0.jld", "data", ls.policy_v0)
save("newpolicyvb.jld", "data", ls.policy_vb)
save("neww_grid.jld", "data", ls.w_grid)
save("newwv0bar.jld", "data", ls_wv0_bar)
save("newwbar.jld", "data", ls_w_bar)

mutable struct Agent
    skill :: Float64 
    wage :: Float64
    employed :: Int
    UI :: Float64   #when determine the range of UI, remember that UI can also be 0 besides the original range
    laidoff :: Int
    search :: Float64
    Pi_s :: Float64
    Unemploymentduration :: Int
end

Agent(employed=0) = Agent(ls.h_grid[1],0,employed,0,0,0,0,0)

Agent()

function new_choice!(ag, ls)
    if ag.employed == 1
        agwi = searchsortedfirst(ls.w_grid, ag.wage)
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        ag.employed = ls.policy_v[agwi,aghi]
        if ag.employed == 0
            ag.laidoff = 1
            ag.UI = 0
        end
    end
end

function welcome_job!(ag, ls)
    ag.wage = rand(ls.w_grid)
    agwi = searchsortedfirst(ls.w_grid, ag.wage)
    aghi = searchsortedfirst(ls.h_grid, ag.skill)
    
    if ag.wage * ag.skill < 0.7 * ag.UI
        agui = searchsortedfirst(ls.I_grid,ag.UI)
        ag.employed = ls.V[agwi,aghi] > ls.Vb[agui,aghi] ? 1 : 0
        if ag.employed == 0
            ag.wage = 0
        else
            ag.search = 0
            ag.Pi_s = 0
            ag.Unemploymentduration = 0
        end
    else
        ag.employed = ls.policy_v[agwi,aghi]
        if ag.employed == 0
            ag.wage = 0
            ag.UI = 0
        else
            ag.search = 0
            ag.Pi_s = 0
            ag.Unemploymentduration = 0
        end
    end
end

function search_hard!(ag, ls)
    if ag.UI > 0 
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        agui = searchsortedfirst(ls.I_grid, ag.UI)
        ag.search = ls.policy_vb[agui,aghi]
        ag.Pi_s = ag.search^0.3
    else
        aghi = searchsortedfirst(ls.h_grid, ag.skill)
        ag.search = ls.policy_v0[aghi]
        ag.Pi_s = ag.search^0.3
    end   
end

num_agents = 10000
sepa_rate = 0.009
agent_indices = collect(1:num_agents)
agents = [Agent() for i=1:num_agents]
sim_length = 1500

unemploymentrate = zeros(sim_length)
unemploymentlow = zeros(sim_length)
unemploymentnormal = zeros(sim_length)

averageud = zeros(sim_length)
LT6unemployment = zeros(sim_length)
LT12unemployment = zeros(sim_length)

proportion_low = zeros(sim_length)
lt6_low = zeros(sim_length)
lt12_low = zeros(sim_length)

avskill = zeros(sim_length)
avwage = zeros(sim_length)
avproductivity = zeros(sim_length)

changedate = 1000

for t in 1:sim_length
    
    if t == 1
        for agent in agents
            agent.skill = ls.h_grid[1]
            agent.employed = 0
            agent.search = ls.policy_v0[1]
            agent.Pi_s = agent.search^0.3
        end
    end
    
    if t == changedate              
        sepa_rate = 0.18
        for agent in agents
            if agent.employed == 0
                agent.skill = ls.h_grid[1]
            end
        end
    end
    
    if t == changedate + 1            
        sepa_rate = 0.009
    end
    
    for agent in agents
        agent.laidoff = 0
    end
    
    employedworker = []
    
    for agent in agents
        if agent.employed == 1
            agent.UI = 0
            agent.Unemploymentduration = 0
            push!(employedworker, agent)
        end
    end
    
    eworker_indices = collect(1:length(employedworker))
    sep_num = round(Int, length(employedworker) * sepa_rate)
    shuffle!(eworker_indices)
    separation_list = eworker_indices[1:sep_num]
    
    for agent in employedworker[separation_list]
        agent.employed = 0
        agent.laidoff = 1
        agent.UI = ls.I_grid[Int(ceil(agent.wage * agent.skill / (2/15)))]
        if t == changedate
            agent.skill = ls.h_grid[1]
        end
    end
    
    stillemployedworker = []
    for agent in agents
        if agent.employed == 1
            push!(stillemployedworker, agent)
        end
    end
    
    stilleworker_indices = collect(1:length(stillemployedworker))
    shuffle!(stilleworker_indices)
    upperskill_num = round(Int, length(stillemployedworker) * 0.1)
    upperskill_list = stilleworker_indices[1:upperskill_num]
    
    for agent in stillemployedworker[upperskill_list]
        if agent.skill == ls.h_grid[21]
            agent.skill = ls.h_grid[21]
        else
            aghi = searchsortedfirst(ls.h_grid, agent.skill)
            agent.skill = ls.h_grid[aghi+1]
        end
    end
    
    for agent in stillemployedworker
        new_choice!(agent,ls)
    end
    
    nojobguys = []
    for agent in agents
        if agent.employed == 0 && agent.laidoff == 0
            agent.Unemploymentduration = agent.Unemploymentduration + 1
            push!(nojobguys, agent)
        end
    end
    
    if length(nojobguys) > 0
        
        nojob_indices = collect(1:length(nojobguys))
        lowerskill_num = round(Int, length(nojobguys) * 0.2)
        shuffle!(nojob_indices)
        lowerskill_list = nojob_indices[1:lowerskill_num]
        
        for agent in nojobguys[lowerskill_list]
            if agent.skill == ls.h_grid[1]
                agent.skill = ls.h_grid[1]
            else
                aghi = searchsortedfirst(ls.h_grid, agent.skill)
                agent.skill = ls.h_grid[aghi-1]
            end
        end
    
    #In this model, the skill of laidoff workers has not been changed
    
        sum_pis = 0
        for agent in nojobguys
            sum_pis = sum_pis + agent.Pi_s
        end
    
        E_Pis = sum_pis / length(nojobguys)
        offer_num = round(Int, length(nojobguys) * E_Pis)
        shuffle!(nojob_indices)
        offer_list = nojob_indices[1:offer_num]
        
        for agent in nojobguys[offer_list]
            welcome_job!(agent, ls)
        end
    end
    
##########################################################
    
    allemployed = []
    for agent in agents
        if agent.employed == 1
            push!(allemployed, agent)
        end
    end
    
    sumproductivity = 0
    sumwage = 0
    for agent in allemployed
        sumproductivity = agent.skill * agent.wage + sumproductivity
        sumwage = agent.wage + sumwage
    end
    avproductivity[t] = sumproductivity / length(allemployed)
    avwage[t] = sumwage / length(allemployed)
    
##########################################################
    lowskill = []
    normalskill = []
 
    for agent in agents
        if agent.skill < ls.h_grid[5]
            push!(lowskill, agent)
        else
           push!(normalskill, agent)
        end
    end
    
    sumskill = 0
    
    for agent in agents
        sumskill = agent.skill + sumskill
    end
    
    avskill[t] = sumskill / length(agents)

    employment = Int[agent.employed for agent in agents]
    unemploymentrate[t] = 1 - mean(employment)

    employmentlow = Int[agent.employed for agent in lowskill]
    unemploymentlow[t] = 1 - mean(employmentlow)

    employmentnormal = Int[agent.employed for agent in normalskill]
    unemploymentnormal[t] = 1 - mean(employmentnormal)

    proportion_low[t] = length(lowskill)/10000
############################################################

    nojob = []
    for agent in agents
        if agent.employed == 0
           push!(nojob, agent)
        end
    end
    
    sumud = 0
    for agent in nojob
        sumud = sumud + agent.Unemploymentduration
    end

    averageud[t] = sumud / length(nojob)

    lt6group = []
    lt12group = []
    
    for agent in agents
        if agent.Unemploymentduration > 11
           push!(lt6group, agent)
        end
        if agent.Unemploymentduration > 23
           push!(lt12group, agent)
        end
    end
    
    lt6low = []
    lt12low = []
    
    for agent in lt6group
        if agent.skill < ls.h_grid[5]
            push!(lt6low, agent)
        end
    end
    
    for agent in lt12group
        if agent.skill < ls.h_grid[5]
            push!(lt12low, agent)
        end
    end
        
    LT6unemployment[t] = length(lt6group) / length(nojob)
    lt6_low[t] = length(lt6low) / length(lt6group)

    LT12unemployment[t] = length(lt12group) / length(nojob)
    lt12_low[t] = length(lt12low) / length(lt12group)

############################################################
    
    newlife_num = round(Int, num_agents * 0.0009)
    shuffle!(agent_indices)
    newlife_list = agent_indices[1:newlife_num]
    
    for agent in agents[newlife_list]
        agent.employed = 0
        agent.UI = 0
        agent.wage = 0
        agent.laidoff = 0
        agent.skill = ls.h_grid[1]
        agent.Unemploymentduration = 0
    end

#############################################################
    
    stillnojob = []
    for agent in agents
        if agent.employed == 0
            push!(stillnojob, agent)
        end
    end
    
    for agent in stillnojob
        search_hard!(agent, ls)
    end      
end

save("new_u.jld", "data", unemploymentrate)
save("new_ulow.jld", "data", unemploymentlow)
save("new_unormal.jld", "data", unemploymentnormal)
save("new_avud.jld", "data", averageud)
save("new_LT6u.jld", "data", LT6unemployment)
save("newLT12u.jld", "data", LT12unemployment)
save("new_proplow.jld", "data", proportion_low)
save("newlt6_low.jld", "data", lt6_low)
save("newlt12_low.jld", "data", lt12_low)
save("newavskill.jld", "data", avskill)
save("newavwage.jld", "data", avwage)
save("newavprod.jld", "data", avproductivity)
